In [13]:
import tensorflow as tf
from tensorflow import keras 
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from keras.datasets import fashion_mnist
import pandas as pd

In [14]:
# Функция для тестирования можелей неройных сетей
def train_model_result(model,data,params={'epochs':15,
                                           'batch_size':None, 
                                           'validation_split':0.2,
                                           'use_multiprocessing' : True,
                                           'verbose':0}  ):
    res= []
    import time
    start = time.time()
    hh=model.fit( data['X_train'],data['y_train'], **params)
    print('Результаты обучения модели')
    history=hh.history
    finish = time.time()
    for key in history:
        res.append((key,history[key][-1]))
        print (key,history[key][-1])
    delta_time = (finish-start)
    print('время работы-', delta_time, 'сек')
    try:
        print('Результаты валидации модели')
        val = model.evaluate( data['X_test'],data['y_test'])
        return res,val,delta_time,history
    except:
        return res,'val',delta_time,history
    finish = time.time()
   
    

In [15]:
# Загрузим данные с датасета 
(X_train,y_train),(X_test,y_test) = fashion_mnist.load_data()

In [16]:
X_train.shape,y_train.shape,X_test.shape,y_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [17]:
X_train.max(),X_train.min()

(255, 0)

In [18]:
# Приведем данные к нужному виду
X_train_w = (X_train / 127) - 1
X_test_w = (X_test / 127) - 1

X_train_w = X_train_w.reshape((-1, 784))
X_test_w = X_test_w.reshape((-1, 784))
y_train_w = to_categorical(y_train)
y_test_w = to_categorical(y_test)

In [19]:
X_train_w.shape,y_train_w.shape,X_test_w.shape,y_test_w.shape

((60000, 784), (60000, 10), (10000, 784), (10000, 10))

### Тестируем сети с раным количеством слоем с общим количесвтом нейронов 1000

In [29]:
# Создадим список с 12 моделей с 2,3,5,10 слоями, с общим количеством нейронов 1000 и разными активю функциями

n=1000
finish_n = y_train_w.shape[1]
models = []
input_dim = X_train_w.shape[1]
hide_levels = [2,3,5,10]
for level in hide_levels:
    n_nueron = (n-finish_n)//(level-1)
    lost_nueron = (n-finish_n)%(level-1)
    if level>2:
        levels_list = [n_nueron+lost_nueron]+[n_nueron for i in range(level-2)]+[finish_n]
    else:
        levels_list =[n-finish_n,finish_n]    
    for fun in['tanh','sigmoid','relu']:
        model_name = f'{level}_level_{"_".join([str(i) for i in levels_list])}_nueron_{fun}'
        denses_list = [Dense(levels_list[0],activation=fun, input_dim=input_dim,name='1')]
        if level>2:
            denses_list+=[ Dense(i,activation=fun,name = str(k+2)) for k,i in enumerate(levels_list[1:-1])] 
        denses_list+=[ Dense(levels_list[-1],activation='softmax',name = str(level))]  
        my_model = Sequential(denses_list,name =model_name )
        models.append((my_model,my_model.get_weights()))
        print(model_name)
print('количесвто моделей - ',len(models))
        

2_level_990_10_nueron_tanh
2_level_990_10_nueron_sigmoid
2_level_990_10_nueron_relu
3_level_495_495_10_nueron_tanh
3_level_495_495_10_nueron_sigmoid
3_level_495_495_10_nueron_relu
5_level_249_247_247_247_10_nueron_tanh
5_level_249_247_247_247_10_nueron_sigmoid
5_level_249_247_247_247_10_nueron_relu
10_level_110_110_110_110_110_110_110_110_110_10_nueron_tanh
10_level_110_110_110_110_110_110_110_110_110_10_nueron_sigmoid
10_level_110_110_110_110_110_110_110_110_110_10_nueron_relu
количесвто моделей -  12


In [30]:
# Активипуем модели
for model in models:
    model[0].compile(
          optimizer='adam',
          loss='categorical_crossentropy',
          metrics=['accuracy'],
                    )

In [31]:
# Зададим парметры и данны для команды fit()

params={'epochs':15,
       'batch_size':None, 
       'validation_split':0.2,
       'use_multiprocessing' : True,
       'verbose':0} 
data = {'X_train':X_train_w,
        'y_train':y_train_w,
        'X_test':X_test_w,
        'y_test':y_test_w}

In [32]:
# Запустим тестирование моделей из ранее созданого списка с разным количесвтом эпох
result=pd.DataFrame(columns=['model','epoch','result_valid'])
for epoh in [15,30,50,100,300]:
    params['epochs']=epoh
    for model in models:
        model[0].set_weights(model[1])
        answer = train_model_result(model = model[0],data=data,params=params)
        result = result.append({'model':model[0].name,'epoch':epoh,'result_valid':answer[1][1]},ignore_index=True)

Результаты обучения модели
loss 0.2905787229537964
accuracy 0.8941875100135803
val_loss 0.44108137488365173
val_accuracy 0.8521666526794434
время работы- 40.00572848320007 сек
Результаты валидации модели
313/313 [==============================] - 0s 761us/step - loss: 0.4718 - accuracy: 0.8481
Результаты обучения модели
loss 0.17562726140022278
accuracy 0.9348124861717224
val_loss 0.3343411386013031
val_accuracy 0.8882499933242798
время работы- 39.89429688453674 сек
Результаты валидации модели
313/313 [==============================] - 0s 783us/step - loss: 0.3592 - accuracy: 0.8799
Результаты обучения модели
loss 0.18601220846176147
accuracy 0.9308333396911621
val_loss 0.37469980120658875
val_accuracy 0.8815000057220459
время работы- 39.73014712333679 сек
Результаты валидации модели
313/313 [==============================] - 0s 745us/step - loss: 0.3933 - accuracy: 0.8805
Результаты обучения модели
loss 0.3061063587665558
accuracy 0.8881250023841858
val_loss 0.3717203736305237
val_acc

313/313 [==============================] - 0s 748us/step - loss: 0.5120 - accuracy: 0.8607
Результаты обучения модели
loss 0.05719529837369919
accuracy 0.9792708158493042
val_loss 0.5027721524238586
val_accuracy 0.8865833282470703
время работы- 133.3703420162201 сек
Результаты валидации модели
313/313 [==============================] - 0s 771us/step - loss: 0.5544 - accuracy: 0.8813
Результаты обучения модели
loss 0.0756109431385994
accuracy 0.9728124737739563
val_loss 0.6560981273651123
val_accuracy 0.8842499852180481
время работы- 132.99299907684326 сек
Результаты валидации модели
313/313 [==============================] - 0s 729us/step - loss: 0.6938 - accuracy: 0.8774
Результаты обучения модели
loss 0.2716011703014374
accuracy 0.9033125042915344
val_loss 0.39717215299606323
val_accuracy 0.8690833449363708
время работы- 126.37697863578796 сек
Результаты валидации модели
313/313 [==============================] - 0s 748us/step - loss: 0.4251 - accuracy: 0.8624
Результаты обучения мод

313/313 [==============================] - 0s 707us/step - loss: 1.0429 - accuracy: 0.8827
Результаты обучения модели
loss 0.054041024297475815
accuracy 0.9917291402816772
val_loss 3.826944589614868
val_accuracy 0.8834999799728394
время работы- 715.0781433582306 сек
Результаты валидации модели
313/313 [==============================] - 0s 662us/step - loss: 4.1120 - accuracy: 0.8798
Результаты обучения модели
loss 0.29102104902267456
accuracy 0.8945208191871643
val_loss 0.4198579490184784
val_accuracy 0.8578333258628845
время работы- 584.3254857063293 сек
Результаты валидации модели
313/313 [==============================] - 0s 780us/step - loss: 0.4460 - accuracy: 0.8517
Результаты обучения модели
loss 0.016708027571439743
accuracy 0.9942499995231628
val_loss 0.8943420648574829
val_accuracy 0.8797500133514404
время работы- 602.4300985336304 сек
Результаты валидации модели
313/313 [==============================] - 0s 774us/step - loss: 0.9225 - accuracy: 0.8788
Результаты обучения мод

In [33]:
# Результат тестирования 12 маоделей с разными слоями на разных эпохах
result.sort_values('result_valid',ascending=False,inplace=True)
result

,model,epoch,result_valid
16,3_level_495_495_10_nueron_sigmoid,30,0.8900
28,3_level_495_495_10_nueron_sigmoid,50,0.8874
59,10_level_110_110_110_110_110_110_110_110_110_1...,300,0.8850
5,3_level_495_495_10_nueron_relu,15,0.8841
35,10_level_110_110_110_110_110_110_110_110_110_1...,50,0.8833
38,2_level_990_10_nueron_relu,100,0.8831
52,3_level_495_495_10_nueron_sigmoid,300,0.8827
40,3_level_495_495_10_nueron_sigmoid,100,0.8825
29,3_level_495_495_10_nueron_relu,50,0.8817
25,2_level_990_10_nueron_sigmoid,50,0.8813


Победила модель 3_level_495_495_10_nueron_sigmoid, три слоя , на сигмоиде,	30 эпох, результат	0.8900
Какой то однозначный вывод о влиянии слоев, эпох сдлеать трудно

### Провреим результаты в двух слойной сети от числа неронов

In [24]:
%%time
N=1000
result2=pd.DataFrame()
for n_nueron in range(10,N+1,50):
    for fun in['tanh','sigmoid','relu']:
        dens =[ Dense(n_nueron,input_dim=X_train_w.shape[1],activation=fun,name= '1'),
                Dense(y_train_w.shape[1],activation='softmax',name='exit')]
        model_level_2 = Sequential(dens, name=f'{n_nueron}_{fun}')
        model_level_2.compile(optimizer='adam',
                              loss='categorical_crossentropy',
                               metrics=['accuracy'])
        a = model_level_2.get_weights()   
                
        for epoh in [15,30,100]:
            model_level_2.set_weights(a)    
            params['epochs']=epoh
            answer = train_model_result(model = model_level_2,data=data,params=params)
            result2 = result2.append({'model':model_level_2.name,'epoch':epoh,'result_valid':answer[1][1]},ignore_index=True)    
                
                
           
    


Результаты обучения модели
loss 0.37762096524238586
accuracy 0.8670833110809326
val_loss 0.4314506947994232
val_accuracy 0.8513333201408386
время работы- 8.331570625305176 сек
Результаты валидации модели
313/313 [==============================] - 0s 438us/step - loss: 0.4646 - accuracy: 0.8424
Результаты обучения модели
loss 0.35361599922180176
accuracy 0.8766250014305115
val_loss 0.4435397684574127
val_accuracy 0.8464999794960022
время работы- 16.274595975875854 сек
Результаты валидации модели
313/313 [==============================] - 0s 438us/step - loss: 0.4710 - accuracy: 0.8402
Результаты обучения модели
loss 0.31467998027801514
accuracy 0.8885416388511658
val_loss 0.5031681656837463
val_accuracy 0.8367499709129333
время работы- 53.92603945732117 сек
Результаты валидации модели
313/313 [==============================] - 0s 432us/step - loss: 0.5364 - accuracy: 0.8320
Результаты обучения модели
loss 0.3814237415790558
accuracy 0.8655208349227905
val_loss 0.4260577857494354
val_acc

313/313 [==============================] - 0s 467us/step - loss: 0.4780 - accuracy: 0.8732
Результаты обучения модели
loss 0.05695357918739319
accuracy 0.9786249995231628
val_loss 0.8312139511108398
val_accuracy 0.8793333172798157
время работы- 69.4311695098877 сек
Результаты валидации модели
313/313 [==============================] - 0s 464us/step - loss: 0.9087 - accuracy: 0.8740
Результаты обучения модели
loss 0.2576204538345337
accuracy 0.9056041836738586
val_loss 0.36813390254974365
val_accuracy 0.8682500123977661
время работы- 11.146141052246094 сек
Результаты валидации модели
313/313 [==============================] - 0s 492us/step - loss: 0.3897 - accuracy: 0.8624
Результаты обучения модели
loss 0.19401516020298004
accuracy 0.9279583096504211
val_loss 0.374379962682724
val_accuracy 0.8764166831970215
время работы- 22.037050247192383 сек
Результаты валидации модели
313/313 [==============================] - 0s 486us/step - loss: 0.4016 - accuracy: 0.8691
Результаты обучения моде

313/313 [==============================] - 0s 1ms/step - loss: 0.9747 - accuracy: 0.8748
Результаты обучения модели
loss 0.2560868263244629
accuracy 0.9051458239555359
val_loss 0.34351402521133423
val_accuracy 0.8767499923706055
время работы- 22.855807781219482 сек
Результаты валидации модели
313/313 [==============================] - 0s 1ms/step - loss: 0.3717 - accuracy: 0.8741
Результаты обучения модели
loss 0.2073584645986557
accuracy 0.9240000247955322
val_loss 0.3689812123775482
val_accuracy 0.8788333535194397
время работы- 44.082120180130005 сек
Результаты валидации модели
313/313 [==============================] - 0s 953us/step - loss: 0.4024 - accuracy: 0.8719
Результаты обучения модели
loss 0.08902604877948761
accuracy 0.9681249856948853
val_loss 0.544916570186615
val_accuracy 0.8725833296775818
время работы- 147.8878779411316 сек
Результаты валидации модели
313/313 [==============================] - 0s 851us/step - loss: 0.6126 - accuracy: 0.8615
Результаты обучения модели
l

313/313 [==============================] - 0s 979us/step - loss: 0.9782 - accuracy: 0.8815
Результаты обучения модели
loss 0.2662879526615143
accuracy 0.9011458158493042
val_loss 0.37764546275138855
val_accuracy 0.871749997138977
время работы- 31.64321231842041 сек
Результаты валидации модели
313/313 [==============================] - 0s 1ms/step - loss: 0.4068 - accuracy: 0.8591
Результаты обучения модели
loss 0.21273382008075714
accuracy 0.9228125214576721
val_loss 0.391466349363327
val_accuracy 0.874666690826416
время работы- 63.20705223083496 сек
Результаты валидации модели
313/313 [==============================] - 0s 902us/step - loss: 0.4190 - accuracy: 0.8676
Результаты обучения модели
loss 0.09266780316829681
accuracy 0.9661250114440918
val_loss 0.5510188937187195
val_accuracy 0.8725833296775818
время работы- 210.81901502609253 сек
Результаты валидации модели
313/313 [==============================] - 0s 991us/step - loss: 0.6207 - accuracy: 0.8641
Результаты обучения модели
l

313/313 [==============================] - 0s 662us/step - loss: 0.3968 - accuracy: 0.8616
Результаты обучения модели
loss 0.2176651805639267
accuracy 0.9196458458900452
val_loss 0.42914196848869324
val_accuracy 0.8614166378974915
время работы- 56.61139702796936 сек
Результаты валидации модели
313/313 [==============================] - 0s 668us/step - loss: 0.4778 - accuracy: 0.8509
Результаты обучения модели
loss 0.10462915897369385
accuracy 0.9621458053588867
val_loss 0.5460857152938843
val_accuracy 0.8744166493415833
время работы- 188.89686059951782 сек
Результаты валидации модели
313/313 [==============================] - 0s 656us/step - loss: 0.6349 - accuracy: 0.8629
Результаты обучения модели
loss 0.16937321424484253
accuracy 0.9372708201408386
val_loss 0.3259114921092987
val_accuracy 0.8900833129882812
время работы- 28.944334268569946 сек
Результаты валидации модели
313/313 [==============================] - 0s 665us/step - loss: 0.3663 - accuracy: 0.8818
Результаты обучения мо

313/313 [==============================] - 0s 608us/step - loss: 0.4591 - accuracy: 0.8499
Результаты обучения модели
loss 0.2303449660539627
accuracy 0.9153333306312561
val_loss 0.39408499002456665
val_accuracy 0.8721666932106018
время работы- 58.16892337799072 сек
Результаты валидации модели
313/313 [==============================] - 0s 608us/step - loss: 0.4249 - accuracy: 0.8620
Результаты обучения модели
loss 0.11602463573217392
accuracy 0.957770824432373
val_loss 0.592604398727417
val_accuracy 0.8680833578109741
время работы- 193.50805616378784 сек
Результаты валидации модели
313/313 [==============================] - 0s 614us/step - loss: 0.6550 - accuracy: 0.8540
Результаты обучения модели
loss 0.1717035323381424
accuracy 0.9365624785423279
val_loss 0.3307693600654602
val_accuracy 0.8888333439826965
время работы- 30.161441564559937 сек
Результаты валидации модели
313/313 [==============================] - 0s 624us/step - loss: 0.3625 - accuracy: 0.8796
Результаты обучения модел

313/313 [==============================] - 0s 672us/step - loss: 0.4318 - accuracy: 0.8547
Результаты обучения модели
loss 0.23741784691810608
accuracy 0.9136041402816772
val_loss 0.4238031804561615
val_accuracy 0.8640000224113464
время работы- 69.39113283157349 сек
Результаты валидации модели
313/313 [==============================] - 0s 652us/step - loss: 0.4478 - accuracy: 0.8601
Результаты обучения модели
loss 0.1199553906917572
accuracy 0.9574375152587891
val_loss 0.5985206365585327
val_accuracy 0.8690000176429749
время работы- 231.50162267684937 сек
Результаты валидации модели
313/313 [==============================] - 0s 649us/step - loss: 0.6478 - accuracy: 0.8622
Результаты обучения модели
loss 0.1721314638853073
accuracy 0.9358333349227905
val_loss 0.3476272523403168
val_accuracy 0.8850833177566528
время работы- 35.26308298110962 сек
Результаты валидации модели
313/313 [==============================] - 0s 694us/step - loss: 0.3866 - accuracy: 0.8745
Результаты обучения модел

In [25]:
result2.sort_values('result_valid',ascending=False,inplace=True)
result2

,epoch,model,result_valid
120,15.0,660_sigmoid,0.8869
94,30.0,510_sigmoid,0.8867
49,30.0,260_sigmoid,0.8867
175,30.0,960_sigmoid,0.8859
107,100.0,560_relu,0.8856
...,...,...,...
1,30.0,10_tanh,0.8402
3,15.0,10_sigmoid,0.8398
5,100.0,10_sigmoid,0.8384
7,30.0,10_relu,0.8377


Здесь лучший результат дает сеть не с самым большим количесвтом нейронов и всего на 15 эпох, но результат немного хуже трехслойной сети выше, с числом тнейронов больше

### Посмотрим результаты лучшей двухслойной сети, на разных оптимизаторах

In [26]:
ful_result=pd.DataFrame()
for fun in['tanh','sigmoid','relu']:
    dens =[ Dense(660,input_dim=X_train_w.shape[1],activation=fun,name= '1'),
            Dense(y_train_w.shape[1],activation='softmax',name='exit')]
    model = Sequential(dens, name=f'660_{fun}')
    a = model.get_weights()
    for epoh in [15,30,100]:
        params['epochs'] = epoh
        for opt in [keras.optimizers.RMSprop(),keras.optimizers.SGD(),keras.optimizers.Nadam()]:       
                model.compile(optimizer=opt,
                              loss='categorical_crossentropy',
                              metrics=['accuracy'])
                model.set_weights(a) 
                answer = train_model_result(model = model,data=data,params=params)
                ful_result = ful_result.append({'model':model.name,\
                                             'epoch':epoh,
                                              'opt':opt,  
                                              'result_valid':answer[1][1]},ignore_index=True)    

Результаты обучения модели
loss 0.23648646473884583
accuracy 0.9152083396911621
val_loss 0.3687364459037781
val_accuracy 0.8790833353996277
время работы- 78.84873843193054 сек
Результаты валидации модели
313/313 [==============================] - 0s 672us/step - loss: 0.3893 - accuracy: 0.8697
Результаты обучения модели
loss 0.2916286587715149
accuracy 0.8964166641235352
val_loss 0.3383161127567291
val_accuracy 0.8767499923706055
время работы- 27.188639640808105 сек
Результаты валидации модели
313/313 [==============================] - 0s 668us/step - loss: 0.3641 - accuracy: 0.8683
Результаты обучения модели
loss 0.26461076736450195
accuracy 0.903124988079071
val_loss 0.3497116267681122
val_accuracy 0.8762500286102295
время работы- 115.3159122467041 сек
Результаты валидации модели
313/313 [==============================] - 0s 672us/step - loss: 0.3895 - accuracy: 0.8665
Результаты обучения модели
loss 0.17082004249095917
accuracy 0.9394375085830688
val_loss 0.4033217430114746
val_accu

313/313 [==============================] - 0s 723us/step - loss: 2.3841 - accuracy: 0.8792
Результаты обучения модели
loss 0.044462911784648895
accuracy 0.9904583096504211
val_loss 0.34467145800590515
val_accuracy 0.8980000019073486
время работы- 185.52178978919983 сек
Результаты валидации модели
313/313 [==============================] - 0s 694us/step - loss: 0.3701 - accuracy: 0.8927
Результаты обучения модели
loss 0.035350240767002106
accuracy 0.9880208373069763
val_loss 1.1094300746917725
val_accuracy 0.8809166550636292
время работы- 770.0736219882965 сек
Результаты валидации модели
313/313 [==============================] - 0s 608us/step - loss: 1.1472 - accuracy: 0.8774


In [38]:
ful_result.sort_values('result_valid',ascending=False,inplace=True)
ful_result

,epoch,model,opt,result_valid
25,100.0,660_relu,<tensorflow.python.keras.optimizer_v2.gradient...,0.8927
7,100.0,660_tanh,<tensorflow.python.keras.optimizer_v2.gradient...,0.8885
23,30.0,660_relu,<tensorflow.python.keras.optimizer_v2.nadam.Na...,0.8859
11,15.0,660_sigmoid,<tensorflow.python.keras.optimizer_v2.nadam.Na...,0.8849
15,100.0,660_sigmoid,<tensorflow.python.keras.optimizer_v2.rmsprop....,0.8847
9,15.0,660_sigmoid,<tensorflow.python.keras.optimizer_v2.rmsprop....,0.8844
12,30.0,660_sigmoid,<tensorflow.python.keras.optimizer_v2.rmsprop....,0.8836
14,30.0,660_sigmoid,<tensorflow.python.keras.optimizer_v2.nadam.Na...,0.8827
22,30.0,660_relu,<tensorflow.python.keras.optimizer_v2.gradient...,0.8815
20,15.0,660_relu,<tensorflow.python.keras.optimizer_v2.nadam.Na...,0.8799


Как видно результат сущетсвенно улучшился при испольловании SGD и relu, при этом понадобилось 100 эпох

### Возьмем сеть 3_level_495_495_10_nueron_sigmoid, которая окозалось лучшей в экпиременте с разным количеством слоев и дала хороший результат 0.89 и попробуем другие оптимизаторы

In [34]:
ful_result2=pd.DataFrame()
for fun in['sigmoid','relu']:
    dens =[ Dense(495,input_dim=X_train_w.shape[1],activation=fun,name= '1'),
            Dense(495,activation=fun,name= '2'),
            Dense(y_train_w.shape[1],activation='softmax',name='exit')]
    model = Sequential(dens, name=f'495_495_10_{fun}')
    a = model.get_weights()
    for epoh in [15,30,100]:
        params['epochs'] = epoh
        for opt in [keras.optimizers.RMSprop(),keras.optimizers.SGD(),keras.optimizers.Nadam()]:       
                model.compile(optimizer=opt,
                              loss='categorical_crossentropy',
                              metrics=['accuracy'])
                model.set_weights(a) 
                answer = train_model_result(model = model,data=data,params=params)
                ful_result2 = ful_result2.append({'model':model.name,\
                                             'epoch':epoh,
                                              'opt':opt,  
                                              'result_valid':answer[1][1]},ignore_index=True)  

Результаты обучения модели
loss 0.18653714656829834
accuracy 0.9328333139419556
val_loss 0.43456950783729553
val_accuracy 0.8866666555404663
время работы- 83.7521984577179 сек
Результаты валидации модели
313/313 [==============================] - 0s 812us/step - loss: 0.4919 - accuracy: 0.8779
Результаты обучения модели
loss 0.446603387594223
accuracy 0.8436874747276306
val_loss 0.44993138313293457
val_accuracy 0.8383333086967468
время работы- 35.19301891326904 сек
Результаты валидации модели
313/313 [==============================] - 0s 764us/step - loss: 0.4757 - accuracy: 0.8274
Результаты обучения модели
loss 0.1637592613697052
accuracy 0.9382500052452087
val_loss 0.322773277759552
val_accuracy 0.893833339214325
время работы- 123.47333645820618 сек
Результаты валидации модели
313/313 [==============================] - 0s 748us/step - loss: 0.3572 - accuracy: 0.8846
Результаты обучения модели
loss 0.10440779477357864
accuracy 0.9651041626930237
val_loss 0.5926728248596191
val_accura

In [35]:
ful_result2.sort_values('result_valid',ascending=False,inplace=True)
ful_result2

,epoch,model,opt,result_valid
13,30.0,495_495_10_relu,<tensorflow.python.keras.optimizer_v2.gradient...,0.8876
16,100.0,495_495_10_relu,<tensorflow.python.keras.optimizer_v2.gradient...,0.8875
2,15.0,495_495_10_sigmoid,<tensorflow.python.keras.optimizer_v2.nadam.Na...,0.8846
6,100.0,495_495_10_sigmoid,<tensorflow.python.keras.optimizer_v2.rmsprop....,0.8846
3,30.0,495_495_10_sigmoid,<tensorflow.python.keras.optimizer_v2.rmsprop....,0.8833
11,15.0,495_495_10_relu,<tensorflow.python.keras.optimizer_v2.nadam.Na...,0.8828
5,30.0,495_495_10_sigmoid,<tensorflow.python.keras.optimizer_v2.nadam.Na...,0.8826
17,100.0,495_495_10_relu,<tensorflow.python.keras.optimizer_v2.nadam.Na...,0.8815
14,30.0,495_495_10_relu,<tensorflow.python.keras.optimizer_v2.nadam.Na...,0.8810
8,100.0,495_495_10_sigmoid,<tensorflow.python.keras.optimizer_v2.nadam.Na...,0.8783


Как видно на других оптимизаторах результат оказался хуже

### Еще раз список всех экспериментов

In [40]:
result.head(10)

,model,epoch,result_valid
16,3_level_495_495_10_nueron_sigmoid,30,0.8900
28,3_level_495_495_10_nueron_sigmoid,50,0.8874
59,10_level_110_110_110_110_110_110_110_110_110_1...,300,0.8850
5,3_level_495_495_10_nueron_relu,15,0.8841
35,10_level_110_110_110_110_110_110_110_110_110_1...,50,0.8833
38,2_level_990_10_nueron_relu,100,0.8831
52,3_level_495_495_10_nueron_sigmoid,300,0.8827
40,3_level_495_495_10_nueron_sigmoid,100,0.8825
29,3_level_495_495_10_nueron_relu,50,0.8817
25,2_level_990_10_nueron_sigmoid,50,0.8813


In [41]:
result2.head(10)

,epoch,model,result_valid
120,15.0,660_sigmoid,0.8869
94,30.0,510_sigmoid,0.8867
49,30.0,260_sigmoid,0.8867
175,30.0,960_sigmoid,0.8859
107,100.0,560_relu,0.8856
115,30.0,610_relu,0.8851
141,15.0,760_relu,0.8850
169,30.0,910_relu,0.8850
178,30.0,960_relu,0.8849
148,30.0,810_sigmoid,0.8846


In [43]:
ful_result.head(10)

,epoch,model,opt,result_valid
25,100.0,660_relu,<tensorflow.python.keras.optimizer_v2.gradient...,0.8927
7,100.0,660_tanh,<tensorflow.python.keras.optimizer_v2.gradient...,0.8885
23,30.0,660_relu,<tensorflow.python.keras.optimizer_v2.nadam.Na...,0.8859
11,15.0,660_sigmoid,<tensorflow.python.keras.optimizer_v2.nadam.Na...,0.8849
15,100.0,660_sigmoid,<tensorflow.python.keras.optimizer_v2.rmsprop....,0.8847
9,15.0,660_sigmoid,<tensorflow.python.keras.optimizer_v2.rmsprop....,0.8844
12,30.0,660_sigmoid,<tensorflow.python.keras.optimizer_v2.rmsprop....,0.8836
14,30.0,660_sigmoid,<tensorflow.python.keras.optimizer_v2.nadam.Na...,0.8827
22,30.0,660_relu,<tensorflow.python.keras.optimizer_v2.gradient...,0.8815
20,15.0,660_relu,<tensorflow.python.keras.optimizer_v2.nadam.Na...,0.8799


In [44]:
ful_result2.head(10)

,epoch,model,opt,result_valid
13,30.0,495_495_10_relu,<tensorflow.python.keras.optimizer_v2.gradient...,0.8876
16,100.0,495_495_10_relu,<tensorflow.python.keras.optimizer_v2.gradient...,0.8875
2,15.0,495_495_10_sigmoid,<tensorflow.python.keras.optimizer_v2.nadam.Na...,0.8846
6,100.0,495_495_10_sigmoid,<tensorflow.python.keras.optimizer_v2.rmsprop....,0.8846
3,30.0,495_495_10_sigmoid,<tensorflow.python.keras.optimizer_v2.rmsprop....,0.8833
11,15.0,495_495_10_relu,<tensorflow.python.keras.optimizer_v2.nadam.Na...,0.8828
5,30.0,495_495_10_sigmoid,<tensorflow.python.keras.optimizer_v2.nadam.Na...,0.8826
17,100.0,495_495_10_relu,<tensorflow.python.keras.optimizer_v2.nadam.Na...,0.8815
14,30.0,495_495_10_relu,<tensorflow.python.keras.optimizer_v2.nadam.Na...,0.8810
8,100.0,495_495_10_sigmoid,<tensorflow.python.keras.optimizer_v2.nadam.Na...,0.8783


Таким образом победитель двухслойная сеть [660,10] , relu,sgd, 100 эпох результат чуть выше чем у catboost (0.8926)

попробуем ее проверить

In [48]:

dens =[ Dense(660,input_dim=X_train_w.shape[1],activation='relu',name= '1'),
            Dense(y_train_w.shape[1],activation='softmax',name='exit')]
model = Sequential(dens, name=f'660_10_relu_100_epohs')
#     a = model.get_weights()
params['epochs'] = 100
     
model.compile(optimizer=keras.optimizers.SGD(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# model.set_weights(a) 
answer = train_model_result(model = model,data=data,params=params)
                

Результаты обучения модели
loss 0.04426822438836098
accuracy 0.9904791712760925
val_loss 0.34044283628463745
val_accuracy 0.8980000019073486
время работы- 168.45625352859497 сек
Результаты валидации модели
313/313 [==============================] - 0s 588us/step - loss: 0.3780 - accuracy: 0.8932


In [49]:
answer

([('loss', 0.04426822438836098),
  ('accuracy', 0.9904791712760925),
  ('val_loss', 0.34044283628463745),
  ('val_accuracy', 0.8980000019073486)],
 [0.37797418236732483, 0.8931999802589417],
 168.45625352859497,
 {'loss': [0.5760420560836792,
   0.43466445803642273,
   0.39848271012306213,
   0.37389805912971497,
   0.35577014088630676,
   0.3415641188621521,
   0.32824230194091797,
   0.3182651996612549,
   0.3079366981983185,
   0.29969221353530884,
   0.29096293449401855,
   0.283912718296051,
   0.2763536274433136,
   0.26924368739128113,
   0.26343438029289246,
   0.25640493631362915,
   0.25052666664123535,
   0.2453811913728714,
   0.23887552320957184,
   0.2335602045059204,
   0.229140967130661,
   0.22318345308303833,
   0.21852311491966248,
   0.2142980396747589,
   0.20996665954589844,
   0.20575840771198273,
   0.20159754157066345,
   0.19724412262439728,
   0.19274243712425232,
   0.1887042373418808,
   0.18524415791034698,
   0.18152444064617157,
   0.17817151546478271,
 

Как видно результат стал еще больше :)